In [30]:
import pickle
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
ACTION_TO_INT = {'UP':0, 'RIGHT':1, 'DOWN':2, 'LEFT':3, 'WAIT':4, 'BOMB':5}

In [31]:
# TODO: check put PyTables to save and load entries of large dictionaries saved as a file
# TODO: Train new tree only after X games
# TODO: Think about how crates positions can be converted to good features

In [32]:
d = dict()
state = np.array([0, 0])
k = tuple(state.tolist())   # convert numpy array to a tuple

d[k] = [[],[]]
d[k][0] = 0
d[k][1] = 1
d[k]

new_state = np.asarray(k)   # convert tuple back to numpy array
new_state

array([0, 0])

In [33]:
with open("my-saved-q-table.pt", "rb") as file:
    q_table:dict = dict(pickle.load(file))
print("q-table entries", len(q_table))

q-table entries 7789


In [34]:
with open("stats.pt", "rb") as file:
    stats:dict = dict(pickle.load(file))
print(stats)

{'n_games': 355026}


In [35]:
def state_to_action_via_lin_val_est(state_str:str):
    temp_str = state_str[1:-1]
    state = np.fromstring(temp_str, dtype=float, sep=' ')
    Q = np.zeros(len(ACTIONS))
    for i, a in enumerate(ACTIONS):
        Q[i] = np.dot(state, action_vectors[a])
    action_int = Q.argmax()
    return ACTIONS[action_int]

In [36]:
c = 0
for key in q_table.keys():
    #print(key)
    action = q_table[key].argmax()
    print(key, "simple_q_learning: ", ACTIONS[action] )
    print(q_table[key] )
    print()

36374 3.54743287 1.02315098]

(-1, 0, -1, 0, 0, 1, -1, 0, -1, 1, 2, 1, 1, -2, 0) simple_q_learning:  LEFT
[1.0022036  1.04812817 0.97887778 1.06149378 1.04886675 1.03220647]

(-1, 1, -1, 0, 0, 1, -1, 0, -1, -2, -1, 1, 1, 0, -1) simple_q_learning:  DOWN
[1.03656707 1.0834358  1.09504379 1.0861573  1.02735682 1.04812656]

(0, -1, 0, 0, 0, 1, 0, -1, 0, -2, 2, 1, 0, 0, -1) simple_q_learning:  BOMB
[1.04326701 1.01677891 1.03246264 1.01017864 1.01636213 1.09049233]

(0, 1, 0, -1, 0, -1, 1, 0, 0, 2, -2, 0, 1, 1, 2) simple_q_learning:  LEFT
[1.0923358  1.02222514 1.03788287 3.55789931 1.00057424 1.05978734]

(1, 1, 0, -1, 0, -1, 0, 0, 0, -2, 1, 0, 1, 1, 1) simple_q_learning:  RIGHT
[1.06658914 1.08990844 1.01409944 1.04096425 1.06468583 1.023024  ]

(0, 1, 0, -1, 0, -1, 0, 0, 0, -2, 2, 0, 1, 1, 2) simple_q_learning:  DOWN
[1.08994053 1.03059572 1.16204595 1.06050011 1.04496688 1.06578673]

(0, 0, 1, -1, 0, -1, 0, 1, 0, 2, 2, 0, 1, -1, -2) simple_q_learning:  DOWN
[1.02033849 1.03993533 1.0973

In [37]:
a = np.array([[1,2,3], [4,5,6]])
a = np.reshape(a, 6)
print("a: ", a)

b = np.array([1, 2, 3])
print("b: ", b)
channels = np.concatenate((a, b))
stacked_channels = np.stack(channels)
stacked_channels.reshape(-1)
stacked_channels


a:  [1 2 3 4 5 6]
b:  [1 2 3]


array([1, 2, 3, 4, 5, 6, 1, 2, 3])

In [95]:
def get_danger_zone(bombs):
    # TODO: sort bombs by countdown (descending)
    field_len = 17
    range:int = 4   #3 tiles in each direction
    danger_zone = np.zeros((field_len, field_len))
    for p, t in sorted (bombs, key = lambda x: x[1], reverse=True):
        danger_zone[ max(p[0]-range,0):min(p[0]+range, field_len) , p[1]] = t
        danger_zone[ p[0], max(p[1]-range,0):min(p[1]+range, field_len)] = t
    return danger_zone

bombs = [((4, 4),3), ((3, 3),1)]
danger_zone = get_danger_zone(bombs)
print(danger_zone)

[[0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 3. 3. 1. 3. 3. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
